In [5]:
import pandas as pd

In [6]:
data = pd.read_csv('dataSetCollection_all_data_45_resources.csv')
data.head()

,_id,name,status,csv,label,parentLabel,prediction,tag,confidence_score,imageAnnotations,remarks,GCStorage_file_path,dataBoost
0,6758b3eb661561b01ca79d02,8c9a7b76-b73e-11ef-aeaf-42010ab60004.jpg,approved,guidezilla<br>gs://auto-ai_resources_fo/projec...,grab,grab,predicted,camera angle => 45,100,"[{""id"":""m4jqaqux"",""name"":""m4jqaqux"",""color"":""r...",NaN,gs://auto-ai_resources_fo/project_6682914be830...,1
1,6758b3ea661561b01ca79cf1,8c989ab8-b73e-11ef-b697-42010ab60004.jpg,approved,shockwave<br>gs://auto-ai_resources_fo/project...,grab,grab,predicted,camera angle => 45,100,"[{""id"":""m4jqc3qt"",""name"":""m4jqc3qt"",""color"":""r...",NaN,gs://auto-ai_resources_fo/project_6682914be830...,1
2,6758b3ea661561b01ca79ce3,8c8f40f8-b73e-11ef-965a-42010ab60004.jpg,approved,shockwave<br>gs://auto-ai_resources_fo/project...,grab,grab,predicted,camera angle => 45,100,"[{""id"":""m4jqd6g4"",""name"":""m4jqd6g4"",""color"":""r...",NaN,gs://auto-ai_resources_fo/project_6682914be830...,1
3,6758b3ea661561b01ca79cf8,8c7be47c-b73e-11ef-be15-42010ab60004.jpg,approved,opticross<br>gs://auto-ai_resources_fo/project...,grab,grab,predicted,camera angle => 45,100,"[{""id"":""m4jqbkt8"",""name"":""m4jqbkt8"",""color"":""r...",NaN,gs://auto-ai_resources_fo/project_6682914be830...,1
4,6758b3ea661561b01ca79cea,8c759946-b73e-11ef-a2e0-42010ab60004.jpg,approved,shockwave<br>gs://auto-ai_resources_fo/project...,grab,grab,predicted,camera angle => 45,100,"[{""id"":""m4jqcnas"",""name"":""m4jqcnas"",""color"":""r...",NaN,gs://auto-ai_resources_fo/project_6682914be830...,1


In [7]:
import pandas as pd
import json
from google.cloud import storage
import os
import cv2
import uuid
from samurai import process_video
from tqdm import tqdm

# Set up Google Cloud credentials
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = './faceopen_key.json'  # Replace with your credentials file
storage_client = storage.Client()
BUCKET_NAME = 'auto-ai_resources_fo'
bucket = storage_client.bucket(BUCKET_NAME)

def process_video_and_coords(row, download_folder):
    """
    Process a single row to download video and extract coordinates
    
    Args:
        row (pd.Series): A single row from the DataFrame
        download_folder (str): Local folder to save downloaded videos
    
    Returns:
        dict: Contains video file path and polygon coordinates
    """
    file_path = row.get('csv', '')
    file_path = file_path.split('>')[1]
    image_annotation = row.get('imageAnnotations', '{}')
    result = {'video_path': None, 'polygon_coords': None}
    
    try:
        # Process video download
        if file_path:
            blob_path = '/'.join(file_path.split('/')[3:])
            print(blob_path)
            blob = bucket.blob(blob_path)
            
            # Generate unique filename to avoid conflicts
            filename = f"{uuid.uuid4()}.mp4"
            filepath = f"{download_folder}/{filename}"
            
            # Create download folder if it doesn't exist
            os.makedirs(download_folder, exist_ok=True)
            
            # Download the file
            blob.download_to_filename(filepath)
            result['video_path'] = filepath
            print(f"Successfully downloaded: {filepath}")
        
        # Process image annotation
        if image_annotation:
            try:
                image_annotation = image_annotation.replace("\'", "\"")
            except:
                print("passed")
                pass
            
            objects = json.loads(str(image_annotation))
            
            print(objects)
            
            for object in objects:
                px = [vertex['x'] for vertex in object['vertices']]
                py = [vertex['y'] for vertex in object['vertices']]
                
                result['polygon_coords'] = {'x': px, 'y': py}
                
                
        
    except Exception as e:
        print(f"Error processing row {row.name}: {str(e)}")
    
    return result

def process_motion_session_with_sam2(video_path, x_coords, y_coords, model_path, output_folder):
    """Process recorded video with SAM2."""

    x1, y1 = min(x_coords), min(y_coords)
    w = max(x_coords) - x1
    h = max(y_coords) - y1

    output_path = os.path.join(output_folder, f"{uuid.uuid4()}.mp4")
    
    motion_type = process_video(
        video_path=video_path,
        coords=(x1, y1, w, h),
        model_path=model_path,
        save_video=True,
        output_path=output_path
    )
    return motion_type

def main():
    # Configuration
    download_folder = 'downloaded_videos'
    csv_file = './dataSetCollection_all_data_45_resources.csv'  # Uploaded file path
    
    # Clear and create download directory
    os.system(f"rm -rf {download_folder}")
    os.system(f"mkdir {download_folder}")
    
    # Read the CSV file
    try:
        df = pd.read_csv(csv_file)
        print(f"Found {len(df)} rows to process")
    except Exception as e:
        print(f"Error reading CSV file: {str(e)}")
        return
    
    # Process rows sequentially
    for _, row in tqdm(df.iterrows(), total=len(df), desc="Processing rows"):
        result = process_video_and_coords(row, download_folder)
        print(result)
        
        filename = result['video_path']
        
        cap = cv2.VideoCapture(filename)
        if not cap.isOpened():
            print("Error: Could not open video.")
            return False
        ret, frame = cap.read()
        if not ret:
            print("Error: Could not read frame.")
            return False
        
        img_path = f"download_folder/{filename.split('/')[1].split('.')[0]}.png"
        cv2.imwrite(img_path, frame)
        cap.release()
        
        try:
            prediction = process_motion_session_with_sam2(
            video_path=filename,
            x_coords=result['polygon_coords']['x'],
            y_coords=result['polygon_coords']['y'],
            model_path='./sam2/checkpoints/sam2.1_hiera_large.pt',
            output_folder='./sam2_results'
        )
        
        except Exception as e:
            print(f"Error processing row {row.name}: {str(e)}")
            prediction = "NA"
            continue
        
        print(prediction)
        
    

main()

Found 163 rows to process


Processing rows:   0%|          | 0/163 [00:00<?, ?it/s]

project_6682914be830ed90533d92b8/model_67519f4d765521b3a575be7b/6758671c2f87c0099317e30e.mp4


Processing rows:   0%|          | 0/163 [00:01<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
# 675a659ae783d0f3a55b809c

